In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install -e git+https://github.com/datamllab/tods.git#egg=tods 
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining tods from git+https://github.com/datamllab/tods.git#egg=tods
  Updating ./src/tods clone
  Running command git fetch -q --tags
  Running command git reset --hard -q d0a5f9d87f6b3cf57b849d8fb8481905b5930bd4
  Attempting uninstall: tods
    Found existing installation: tods 0.0.2
    Can't uninstall 'tods'. No files were found to uninstall.
  Running setup.py develop for tods


In [ ]:
import sys
import argparse
import os
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn import metrics
from d3m import index
from d3m.metadata.base import ArgumentType
from d3m.metadata.pipeline import Pipeline, PrimitiveStep
from axolotl.backend.simple import SimpleRunner
from tods import generate_dataset, generate_problem
from tods.searcher import BruteForceSearch
from tods import generate_dataset, load_pipeline, evaluate_pipeline
from tods.sk_interface.detection_algorithm.DeepLog_skinterface import DeepLogSKI
from tods.sk_interface.detection_algorithm.Telemanom_skinterface import TelemanomSKI


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load TRAIN / TEST data from drive
#labelDF = pd.read_csv('/content/drive/MyDrive/all_flights_detail_label2.csv', header=None)
labelDF = pd.read_csv('/content/drive/MyDrive/speed_t4013.csv')

In [ ]:
#labelDF=labelDF.iloc[:,0:2]
labelDF['timestamp'] = pd.to_datetime(labelDF['timestamp'])
labelDF['timestamp'] = (labelDF['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
labelDF = labelDF.drop(labelDF.columns[[5, 6, 7]], axis=1)

In [ ]:
labelDF.tail()

,timestamp,value,label
2490,1442505540,61,0
2491,1442505840,66,0
2492,1442506140,65,0
2493,1442506440,65,0
2494,1442506740,60,0


In [ ]:
group1 = new_labels[new_labels[0] == 0]
group1 = group1.iloc[:,0:3]
group2 = new_labels[new_labels[0] == 1]
group2 = group2.iloc[:,0:3]
#group3 = new_labels[new_labels[0] == 2]
#group3 = group3.iloc[:,0:3]

In [ ]:
group1.head()

,timestamp,value,label
0,1441106700,58,0
1,1441107000,63,0
2,1441107300,63,0
3,1441107600,64,0
4,1441108500,58,0


In [ ]:
group1=labelDF.iloc[0:1999]
group2=labelDF.iloc[2000:3999]
group3=labelDF.iloc[4000:5999]
group4=labelDF.iloc[6000:7999]
group5=labelDF.iloc[8000:9999]

In [ ]:
group2.reset_index(drop=True, inplace=True)

In [ ]:
import pandas as pd

from axolotl.backend.simple import SimpleRunner

from tods import generate_dataset, generate_problem
from tods.searcher import BruteForceSearch

# Some information
#table_path = 'datasets/yahoo_sub_5.csv'
target_index = 2 # what column is the target
time_limit = 300 # How many seconds you wanna search
metric = 'F1_MACRO' # F1 on both label 0 and 1

# Read data and generate dataset and problem
#df = pd.read_csv(table_path)
dataset = generate_dataset(group2, target_index=target_index)
problem_description = generate_problem(dataset, metric)

# Start backend
backend = SimpleRunner(random_seed=0)

# Start search algorithm
search = BruteForceSearch(problem_description=problem_description,
                          backend=backend)

# Find the best pipeline
best_runtime, best_pipeline_result = search.search_fit(input_data=[dataset], time_limit=time_limit)
best_pipeline = best_runtime.pipeline
best_output = best_pipeline_result.output

# Evaluate the best pipeline
best_scores = search.evaluate(best_pipeline).scores

Not all provided hyper-parameters for the data preparation pipeline 79ce71bd-db96-494b-a455-14f2e2ac5040 were used: ['method', 'number_of_folds', 'randomSeed', 'shuffle', 'stratified']
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_382 (Dense)            (None, 4)                 20        
_________________________________________________________________
dropout_255 (Dropout)        (None, 4)                 0         
_________________________________________________________________
dense_383 (Dense)            (None, 4)                 20        
_________________________________________________________________
dropout_256 (Dropout)        (None, 4)                 0         
_________________________________________________________________
dense_384 (Dense)            (None, 1)                 5         
_________________________________________________________________
dropout_257 (Dropout)        (None, 1)                 0         
_________________________________________________________________
dense_385 (Dense)            (None, 4)               

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1668/1668 [==============================] - 7s 4ms/sample - loss: 1.5104 - val_loss: 1.3168
Epoch 2/20
1668/1668 [==============================] - 0s 109us/sample - loss: 1.4214 - val_loss: 1.2627
Epoch 3/20
1668/1668 [==============================] - 0s 108us/sample - loss: 1.3581 - val_loss: 1.2137
Epoch 4/20
1668/1668 [==============================] - 0s 113us/sample - loss: 1.3063 - val_loss: 1.1679
Epoch 5/20
1668/1668 [==============================] - 0s 115us/sample - loss: 1.2544 - val_loss: 1.1227
Epoch 6/20
1668/1668 [==============================] - 0s 114us/sample - loss: 1.2048 - val_loss: 1.0764
Epoch 7/20
1668/1668 [==============================] - 0s 117us/sample - loss: 1.1611 - val_loss: 1.0389
Epoch 8/20
1668/1668 [==============================] - 0s 116us/sample - loss: 1.1241 - val_loss: 1.0072
Epoch 9/20
1668/1668 [==============================] - 0s 113us/sample - loss: 1.0924 - val_loss: 0.9806
Epoch 10/20
1668/1668 [==============================] - 0s

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
Traceback (most recent call last):
  File "/content/src/tods/tods/searcher/brute_force_search.py", line 62, in _search
    for error in pipeline_result.error:
TypeError: 'NoneType' object is not iterable
Not all provided hyper-parameters for the data preparation pipeline 79ce71bd-db96-494b-a455-14f2e2ac5040 were used: ['method', 'number_of_folds', 'randomSeed', 'shu

...

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
Not all provided hyper-parameters for the data preparation pipeline 79ce71bd-db96-494b-a455-14f2e2ac5040 were used: ['method', 'number_of_folds', 'randomSeed', 'shuffle', 'stratified']
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_463 (Dense)            (None, 4)                 20        
_________________________________________________________________
dropout_313 (Dropout)        (None, 4)                 0         
_________________________________________________________________
dense_464 (Dense)            (None, 4)                 20        
_________________________________________________________________
dropout_314 (Dropout)        (None, 4)                 0         
_________________________________________________________________
dense_465 (Dense)            (None, 1)                 5         
_________________________________________________________________
dropout_315 (Dropout)        (None, 1)                 0         
_________________________________________________________________
dense_466 (Dense)            (None, 4)               

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1668/1668 [==============================] - 8s 5ms/sample - loss: 1.4093 - val_loss: 1.1816
Epoch 2/20
1668/1668 [==============================] - 0s 121us/sample - loss: 1.3729 - val_loss: 1.1540
Epoch 3/20
1668/1668 [==============================] - 0s 121us/sample - loss: 1.3419 - val_loss: 1.1288
Epoch 4/20
1668/1668 [==============================] - 0s 118us/sample - loss: 1.3131 - val_loss: 1.1054
Epoch 5/20
1668/1668 [==============================] - 0s 118us/sample - loss: 1.2865 - val_loss: 1.0833
Epoch 6/20
1668/1668 [==============================] - 0s 118us/sample - loss: 1.2637 - val_loss: 1.0621
Epoch 7/20
1668/1668 [==============================] - 0s 124us/sample - loss: 1.2396 - val_loss: 1.0421
Epoch 8/20
1668/1668 [==============================] - 0s 122us/sample - loss: 1.2168 - val_loss: 1.0231
Epoch 9/20
1668/1668 [==============================] - 0s 122us/sample - loss: 1.1968 - val_loss: 1.0051
Epoch 10/20
1668/1668 [==============================] - 0s

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [ ]:
print('Search History:')
for pipeline_result in search.history:
    print('-' * 52)
    print('Pipeline id:', pipeline_result.pipeline.id)
    print(pipeline_result.scores)

Search History:
----------------------------------------------------
Pipeline id: 702fc6d7-e01c-4986-bde4-081ca1554138
     metric     value  normalized  randomSeed  fold
0  F1_MACRO  0.592933    0.592933           0     0
----------------------------------------------------
Pipeline id: 3b7664a4-162c-4118-a97e-39c576135187
     metric     value  normalized  randomSeed  fold
0  F1_MACRO  0.545093    0.545093           0     0
----------------------------------------------------
Pipeline id: 80321adf-9b87-4a71-9675-784c7d9d3958
     metric     value  normalized  randomSeed  fold
0  F1_MACRO  0.508459    0.508459           0     0
----------------------------------------------------
Pipeline id: b77223ff-1d29-49ee-998a-5512f7d50654
     metric     value  normalized  randomSeed  fold
0  F1_MACRO  0.497254    0.497254           0     0
----------------------------------------------------
Pipeline id: d1c4036e-c7df-462f-827b-479a64726d9a
     metric     value  normalized  randomSeed  fold
0

In [ ]:
print('Best pipeline:')
print('-' * 52)
print('Pipeline id:', best_pipeline.id)
print('Pipeline json:', best_pipeline.to_json())
print('Output:')
print(best_output)
print('Scores:')
print(best_scores)

Best pipeline:
----------------------------------------------------
Pipeline id: 702fc6d7-e01c-4986-bde4-081ca1554138
Pipeline json: {"id": "702fc6d7-e01c-4986-bde4-081ca1554138", "schema": "https://metadata.datadrivendiscovery.org/schemas/v0/pipeline.json", "created": "2022-06-15T13:58:24.250205Z", "inputs": [{"name": "inputs"}], "outputs": [{"data": "steps.7.produce", "name": "output predictions"}], "steps": [{"type": "PRIMITIVE", "primitive": {"id": "c78138d9-9377-31dc-aee8-83d9df049c60", "version": "0.3.0", "python_path": "d3m.primitives.tods.data_processing.dataset_to_dataframe", "name": "Extract a DataFrame from a Dataset"}, "arguments": {"inputs": {"type": "CONTAINER", "data": "inputs.0"}}, "outputs": [{"id": "produce"}]}, {"type": "PRIMITIVE", "primitive": {"id": "81235c29-aeb9-3828-911a-1b25319b6998", "version": "0.6.0", "python_path": "d3m.primitives.tods.data_processing.column_parser", "name": "Parses strings into their types"}, "arguments": {"inputs": {"type": "CONTAINER", 